In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [59]:
import pandas as pd

df = pd.read_csv("List_Of_Companies.csv")
df.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,6-Oct-08,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,EQ,3-May-95,10,1,INE253B01015,10
2,360ONE,360 ONE WAM LIMITED,EQ,19-Sep-19,1,1,INE466L01038,1
3,3IINFOLTD,3i Infotech Limited,EQ,22-Oct-21,10,1,INE748C01038,10
4,3MINDIA,3M India Limited,EQ,13-Aug-04,10,1,INE470A01017,10


In [11]:
company_symbols = df["SYMBOL"].tolist()

In [32]:
def fetch_company_page(company_symbol):
    url = f"https://www.screener.in/company/{company_symbol}/"
    response = requests.get(url)
    
    if response.status_code != 200:
        return None
    
    return BeautifulSoup(response.text, "html.parser")



In [47]:
def extract_eps(soup):
    # Find all table rows
    for row in soup.select("tr"):
        cells = row.find_all("td")
        
        if len(cells) > 1:
            metric_name = cells[0].text.strip()
            
            if metric_name == "EPS in Rs":
                # Latest value is the LAST column
                return cells[-1].text.strip()
    
    return None


In [33]:
def extract_ratios(soup, required_ratios):
    data = {}
    
    for li in soup.select("li"):
        name = li.find("span", class_="name")
        value = li.find("span", class_="number")
        
        if name and value:
            ratio_name = name.text.strip()
            if ratio_name in required_ratios:
                data[ratio_name] = value.text.strip()
    
    return data


In [ ]:
def fetch_fundamentals(company_symbol):
    soup = fetch_company_page(company_symbol)
    
    if soup is None:
        return None
    
    ratios_needed = [
        "ROE",
        "ROCE",
        "Dividend Yield",
        "Market Cap",
        "Stock P/E"
    ]
    
    ratios = extract_ratios(soup, ratios_needed)
    eps = extract_eps(soup)
    
    return {
        "Company": company_symbol,
        **ratios,
        "EPS": eps
    }


In [64]:
results= []

for i, company in enumerate(company_symbols):
    fundamentals = fetch_fundamentals(company)
    results_add.append(fundamentals)
    print(f"{i+1}/{len(company_symbols)} - {company}")
    time.sleep(2)


1/2099 - 20MICRONS
2/2099 - 21STCENMGM
3/2099 - 360ONE
4/2099 - 3IINFOLTD
5/2099 - 3MINDIA
6/2099 - 3PLAND
7/2099 - 5PAISA


KeyboardInterrupt: 

In [65]:
df_fundamentals= pd.DataFrame(results)

In [67]:
df_fundamentals.head()

""


In [68]:
df_fundamentals.to_csv("nse_fundamental_ratios.csv", index=False)
